In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


In [4]:
df = pd.read_csv('/content/Invisalign_final_docx (3).csv', encoding='ISO-8859-1')

In [5]:
df

,SubmissionNumber,Cleaned_Text,final_text,topic,Option 1
0,1,feeling trays mouth never throughout entire pr...,Did any of you ever get used to the feeling of...,0,Pain and Oral Dysfunction
1,1,treatment still prefer trays never thought wou...,I've been done with treatment for a year now a...,2,NaN
2,1,still constantly theyre filling saliva,I'm on tray 29 and STILL suck on them. They co...,3,Pain and Oral Dysfunction
3,1,NaN,I__ÑÐ²m more used to having them in than out lol,0,NaN
4,1,feels comfy,This. Feels comfy with them in,3,NaN
...,...,...,...,...,...
10852,437,practice brazilian jiujitsu class people usual...,From my understanding this is relatively norma...,4,NaN
10853,438,theyre looking great youre halfway collecting ...,No I don_ÐÐ__ÐÑ__ÐÐt clench my teeth.\n ...,2,NaN
10854,438,worse teeth initial trays trays first round te...,I did notice that my bite changed (negatively)...,2,NaN
10855,438,sorry quality photos first compared investment...,"Yeah, I think that_ÐÐ__ÐÑ__ÐÐs a pretty ...",1,NaN


In [6]:
df1=df.iloc[:4000,]

In [7]:
df1.isnull().sum()

SubmissionNumber       0
Cleaned_Text         117
final_text             0
topic                  0
Option 1            1745
dtype: int64

In [8]:
df1

,SubmissionNumber,Cleaned_Text,final_text,topic,Option 1
0,1,feeling trays mouth never throughout entire pr...,Did any of you ever get used to the feeling of...,0,Pain and Oral Dysfunction
1,1,treatment still prefer trays never thought wou...,I've been done with treatment for a year now a...,2,NaN
2,1,still constantly theyre filling saliva,I'm on tray 29 and STILL suck on them. They co...,3,Pain and Oral Dysfunction
3,1,NaN,I__ÑÐ²m more used to having them in than out lol,0,NaN
4,1,feels comfy,This. Feels comfy with them in,3,NaN
...,...,...,...,...,...
3995,167,sometimes cheap twicex009d online providers ex...,I went through a dentist as well & I agree fir...,2,Pain and Oral Dysfunction
3996,167,would expensive option heard people anyway,I don't go back until June 9th. I'll update then.,2,NaN
3997,167,afford please regular braces,What did they do to fix the bite?,2,Mealtime and Eating
3998,167,better waiting adult ortho dental insurance qu...,"I've had invisalign since December, 12 trays s...",2,Pain and Oral Dysfunction


In [9]:
df1['Option 1'].fillna("Others",inplace=True)
df1['Option 1'].value_counts()

Others                       1745
Appearance                    587
Pain and Oral Dysfunction     398
Tray Time                     386
Oral Hygiene                  353
Finance                       338
Mealtime and Eating           193
Name: Option 1, dtype: int64

In [10]:
df1=df1[df1['Option 1']!='Others']
df1.shape

(2255, 5)

In [11]:
df1=df1.drop(columns=['topic','Cleaned_Text'])
df1

,SubmissionNumber,final_text,Option 1
0,1,Did any of you ever get used to the feeling of...,Pain and Oral Dysfunction
2,1,I'm on tray 29 and STILL suck on them. They co...,Pain and Oral Dysfunction
9,1,I__ÑÐ²m on tray 10 and I feel uncomfortable b...,Pain and Oral Dysfunction
10,1,"Totally understanding the lip thing, it would ...",Pain and Oral Dysfunction
13,1,omg it made me want to grind them when I had t...,Pain and Oral Dysfunction
...,...,...,...
3994,167,>The spaces between my molars don't bother me ...,Mealtime and Eating
3995,167,I went through a dentist as well & I agree fir...,Pain and Oral Dysfunction
3997,167,What did they do to fix the bite?,Mealtime and Eating
3998,167,"I've had invisalign since December, 12 trays s...",Pain and Oral Dysfunction


In [12]:
# sampled_df = pd.DataFrame()

# # Define the number of data points you want to sample per label
# sample_size = 300

# # Loop through each unique label in the 'label' column
# for label in df1['Option 1'].unique():
#     # Filter the DataFrame to select only rows with the current label
#     label_df = df1[df1['Option 1'] == label]

#     # Sample 'sample_size' data points from the current label
#     if len(label_df) >= sample_size:
#         sampled_label_df = label_df.sample(sample_size)
#     else:
#         # If there are fewer than 'sample_size' data points for this label, take all available data points
#         sampled_label_df = label_df

#     # Append the sampled data to the 'sampled_df' DataFrame
#     sampled_df = pd.concat([sampled_df, sampled_label_df])

# # Reset the index of the sampled DataFrame
# sampled_df.reset_index(drop=True, inplace=True)

# # Now 'sampled_df' contains 300 data points from each label
# sampled_df.shape
#df1=sampled_df.copy()



In [13]:
df1=df1[df1['final_text']!=1]

In [14]:
### Remove comments less than 3 letter words

def count_words(text):
    x=[i for i in text.split(" ")]
    if len(x) < 5:
        return np.nan
    return text
df1['final_text']=df1['final_text'].apply(count_words)

In [15]:
df1.dropna(inplace=True)

In [16]:
np.round(100 * (df1['Option 1'].value_counts()/df1['Option 1'].shape[0]),1).astype('str') + '%'

Appearance                   25.7%
Pain and Oral Dysfunction    17.7%
Tray Time                    17.3%
Oral Hygiene                 15.6%
Finance                      15.1%
Mealtime and Eating           8.6%
Name: Option 1, dtype: object

In [17]:
## Data Cleaning
def clean_text(text):
    if isinstance(text, str):
    # Remove all non-alphanumeric characters and extra spaces
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]+', '', text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        return cleaned_text.strip()
    else:
        return ''
# Apply the clean_text function to the 'Text' column
df1['Cleaned_Text'] = df1['final_text'].apply(clean_text)
df1

,SubmissionNumber,final_text,Option 1,Cleaned_Text
0,1,Did any of you ever get used to the feeling of...,Pain and Oral Dysfunction,Did any of you ever get used to the feeling of...
2,1,I'm on tray 29 and STILL suck on them. They co...,Pain and Oral Dysfunction,Im on tray 29 and STILL suck on them They cons...
9,1,I__ÑÐ²m on tray 10 and I feel uncomfortable b...,Pain and Oral Dysfunction,Im on tray 10 and I feel uncomfortable both in...
10,1,"Totally understanding the lip thing, it would ...",Pain and Oral Dysfunction,Totally understanding the lip thing it would l...
13,1,omg it made me want to grind them when I had t...,Pain and Oral Dysfunction,omg it made me want to grind them when I had t...
...,...,...,...,...
3994,167,>The spaces between my molars don't bother me ...,Mealtime and Eating,The spaces between my molars dont bother me ex...
3995,167,I went through a dentist as well & I agree fir...,Pain and Oral Dysfunction,I went through a dentist as well I agree first...
3997,167,What did they do to fix the bite?,Mealtime and Eating,What did they do to fix the bite
3998,167,"I've had invisalign since December, 12 trays s...",Pain and Oral Dysfunction,Ive had invisalign since December 12 trays swi...


import nltk
nltk.download('stopwords')

import nltk
nltk.download('punkt')

import nltk
nltk.download('wordnet')

import nltk
nltk.download('omw-1.4')

In [18]:
# remove stop words
# Assuming df1 is your DataFrame with a column 'stemmed_text'
# Create a list of stop words up to three letters
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words = [word.lower() for word in stop_words if len(word) > 3]

# Remove stop words from the 'stemmed_text' column
df1['Cleaned_Text']= df1['Cleaned_Text'].str.lower()
df1['Cleaned_Text'] =df1['Cleaned_Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Print the updated DataFrame
df1.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,SubmissionNumber,final_text,Option 1,Cleaned_Text
0,1,Did any of you ever get used to the feeling of...,Pain and Oral Dysfunction,did any of you ever get used to the feeling of...
2,1,I'm on tray 29 and STILL suck on them. They co...,Pain and Oral Dysfunction,im on tray 29 and still suck on constantly fee...
9,1,I__ÑÐ²m on tray 10 and I feel uncomfortable b...,Pain and Oral Dysfunction,im on tray 10 and i feel uncomfortable in and ...
10,1,"Totally understanding the lip thing, it would ...",Pain and Oral Dysfunction,totally understanding the lip thing it would l...
13,1,omg it made me want to grind them when I had t...,Pain and Oral Dysfunction,omg it made me want to grind i had on


In [19]:
### Lemmatization
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word, wordnet.VERB) for word in words]
    return ' '.join(lemmatized_words)

# Apply lemmatization to the text column
df1['lemmatized_text'] = df1['Cleaned_Text'].apply(lemmatize_text)

# Print the DataFrame with lemmatized text
df1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,SubmissionNumber,final_text,Option 1,Cleaned_Text,lemmatized_text
0,1,Did any of you ever get used to the feeling of...,Pain and Oral Dysfunction,did any of you ever get used to the feeling of...,do any of you ever get use to the feel of the ...
2,1,I'm on tray 29 and STILL suck on them. They co...,Pain and Oral Dysfunction,im on tray 29 and still suck on constantly fee...,im on tray 29 and still suck on constantly fee...
9,1,I__ÑÐ²m on tray 10 and I feel uncomfortable b...,Pain and Oral Dysfunction,im on tray 10 and i feel uncomfortable in and ...,im on tray 10 and i feel uncomfortable in and ...
10,1,"Totally understanding the lip thing, it would ...",Pain and Oral Dysfunction,totally understanding the lip thing it would l...,totally understand the lip thing it would look...
13,1,omg it made me want to grind them when I had t...,Pain and Oral Dysfunction,omg it made me want to grind i had on,omg it make me want to grind i have on
...,...,...,...,...,...
3994,167,>The spaces between my molars don't bother me ...,Mealtime and Eating,the spaces my molars dont bother me except get...,the space my molars dont bother me except get ...
3995,167,I went through a dentist as well & I agree fir...,Pain and Oral Dysfunction,i went a dentist as well i agree first wrong m...,i go a dentist as well i agree first wrong mov...
3997,167,What did they do to fix the bite?,Mealtime and Eating,did do to fix the bite,do do to fix the bite
3998,167,"I've had invisalign since December, 12 trays s...",Pain and Oral Dysfunction,ive had invisalign since december 12 trays swi...,ive have invisalign since december 12 trays sw...


In [20]:
df1.to_csv('df1.csv', index=False)

#DistilBERT

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.2 MB/s eta 0:00:00


In [23]:
import transformers
print(transformers.__version__)

4.33.2


#DistilBert

In [25]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np

# check CUDA device availability
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# load the data
data_path = "/content/df1.csv"
df = pd.read_csv(data_path)

# label encoding
label_encoder = LabelEncoder()
df['encoded_labels'] = label_encoder.fit_transform(df['Option 1'])

# train/test split
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['lemmatized_text'], df['encoded_labels'], test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.2, random_state=42)

#initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# convert text to token
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)


# generate Pytorch dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels.tolist())
val_dataset = CustomDataset(val_encodings, val_labels.tolist())
test_dataset = CustomDataset(test_encodings, test_labels.tolist())

# model initialize
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_)).to(device) # 모델을 CUDA 장치로 옮깁니다.

# function to calculate accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.0077,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate(test_dataset)

There are 1 GPU(s) available.
Device name: Tesla T4


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
10,1.805600,1.800425,0.132022
20,1.796500,1.792576,0.143258
30,1.793500,1.782134,0.213483
40,1.772400,1.769649,0.241573
50,1.757900,1.756219,0.250000
60,1.737900,1.742260,0.255618


Step,Training Loss,Validation Loss,Accuracy
10,1.805600,1.800425,0.132022
20,1.796500,1.792576,0.143258
30,1.793500,1.782134,0.213483
40,1.772400,1.769649,0.241573
50,1.757900,1.756219,0.250000
60,1.737900,1.742260,0.255618
70,1.771300,1.729670,0.255618
80,1.717600,1.716380,0.258427
90,1.715900,1.698410,0.266854
100,1.719900,1.671202,0.314607


{'eval_loss': 0.9593077898025513,
 'eval_accuracy': 0.7222222222222222,
 'eval_runtime': 0.6616,
 'eval_samples_per_second': 136.025,
 'eval_steps_per_second': 9.068,
 'epoch': 3.0}